# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import HTML
from itertools import combinations

PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


/home/eddyod/programming/pipeline/src


In [2]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import apply_affine_transform, get_affine_transformation, list_coms, \
    compute_affine_transformation, compute_affine_transformation_centroid
from library.atlas.brain_structure_manager import BrainStructureManager
from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
read_image, write_image


xgboost version=2.1.4


In [3]:
def absolute_sum(l):
    la = np.array(l)
    nabs = (np.array(la*la))
    return np.sum(la, axis=0)

def sum_square_com(com):
    ss = np.sqrt(sum([s*s for s in com]))
    return ss

In [21]:
atlas_all = list_coms('AtlasV7')
allen_all = list_coms('Allen')
common_keys = list(atlas_all.keys() & allen_all.keys())
atlas_common = np.array([atlas_all[s] for s in common_keys])
allen_common = np.array([allen_all[s] for s in common_keys])
print(len(common_keys))

37


In [24]:
# these keys have very high errors: 'RtTg', 'AP'
bad_keys = ('RtTg', 'AP')
good_keys = set(common_keys) - set(bad_keys)
print(f'#good_keys={len(good_keys)}')

#good_keys=35


In [25]:
atlas_src = np.array([atlas_all[s] for s in good_keys])
allen_src = np.array([allen_all[s] for s in good_keys])
transformation_matrix = compute_affine_transformation(atlas_src, allen_src)

In [26]:
df_list = []
error = []
transformed_dict = {}
for structure in common_keys:
    atlas0 = np.array(atlas_all[structure])
    allen0 = np.array(allen_all[structure]) 
    transformed = apply_affine_transform(atlas0, transformation_matrix)
    transformed = [x for x in transformed]
    difference = [a - b for a, b in zip(transformed, allen0)]
    ss = sum_square_com(difference)
    row = [structure, np.round(atlas0), np.round(allen0), 
           np.round(transformed), np.round(difference), ss]
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
print('RMS', sum(error)/len(df_list))
# test keys RMS 25.61709479307028
# base keys RMS 25.602246865972344

RMS 250.58580923794403


In [27]:
structure = 'SC'
print(f'Atlas {structure} {np.round(np.array(atlas_all[structure]))}')
print(f'Allen {structure} {np.round(np.array(allen_all[structure]))}')
print(f'Atlas transformed {structure} {np.round(np.array(transformed_dict[structure]))}')

Atlas SC [7828. 2579. 5730.]
Allen SC [9140. 2388. 5692.]
Atlas transformed SC [9185. 2305. 5716.]


In [28]:
columns = ['structure', 'atlas0', 'allen0', 'transformed', 'difference', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(2)
df.sort_values(by=['sumsquares'], inplace=True)
HTML(df.to_html(index=False))

structure,atlas0,allen0,transformed,difference,sumsquares
LRt_L,"[10985.0, 6472.0, 4571.0]","[12344.0, 6991.0, 4393.0]","[12371.0, 6990.0, 4415.0]","[26.0, -1.0, 22.0]",34.30
LRt_R,"[10985.0, 6472.0, 6829.0]","[12344.0, 6991.0, 6993.0]","[12377.0, 7005.0, 6933.0]","[33.0, 14.0, -59.0]",69.18
SNC_R,"[6780.0, 4826.0, 6921.0]","[8356.0, 5123.0, 7092.0]","[8365.0, 5029.0, 7071.0]","[9.0, -94.0, -21.0]",96.49
SC,"[7828.0, 2579.0, 5730.0]","[9140.0, 2388.0, 5692.0]","[9185.0, 2305.0, 5716.0]","[44.0, -83.0, 23.0]",96.82
5N_R,"[8668.0, 5020.0, 7061.0]","[10193.0, 5286.0, 7293.0]","[10130.0, 5260.0, 7207.0]","[-63.0, -26.0, -85.0]",109.29
5N_L,"[8668.0, 5020.0, 4339.0]","[10193.0, 5287.0, 4092.0]","[10123.0, 5241.0, 4173.0]","[-70.0, -45.0, 81.0]",116.24
IC,"[9084.0, 2510.0, 5700.0]","[10400.0, 2325.0, 5675.0]","[10346.0, 2218.0, 5667.0]","[-54.0, -107.0, -8.0]",119.87
SNC_L,"[6780.0, 4826.0, 4479.0]","[8354.0, 5125.0, 4294.0]","[8358.0, 5012.0, 4349.0]","[4.0, -112.0, 56.0]",125.38
3N_L,"[7737.0, 3916.0, 5553.0]","[9102.0, 3794.0, 5521.0]","[9188.0, 3918.0, 5529.0]","[86.0, 124.0, 8.0]",150.54
3N_R,"[7737.0, 3916.0, 5847.0]","[9103.0, 3794.0, 5864.0]","[9189.0, 3920.0, 5857.0]","[86.0, 126.0, -8.0]",152.42


In [9]:
df['sumsquares'].sum()/len(df.index)
# 259.96675675675675
# 252.11243243243237

252.11243243243243

['LRt_L', 'SC', 'LRt_R', 'IC', '5N_R', '5N_L', 'SNC_R', '6N_L', '3N_L', '3N_R', '6N_R', 'SNC_L', 'PBG_R', '4N_L', 'PBG_L', '4N_R', 'Sp5I_R', 'VLL_L', '7N_R', 'VLL_R', '7N_L', 'Sp5I_L', 'Amb_R']


In [18]:
for threshold in range(100,550, 25):
    good_keys = df.query(f'sumsquares < {threshold}')['structure']
    good_keys = good_keys.to_list()
    atlas_src = np.array([atlas_all[s] for s in good_keys])
    allen_src = np.array([allen_all[s] for s in good_keys])
    transformation_matrix = compute_affine_transformation(atlas_src, allen_src)
    error = []
    for structure in common_keys:
        atlas0 = np.array(atlas_all[structure])
        allen0 = np.array(allen_all[structure]) 
        transformed = apply_affine_transform(atlas0, transformation_matrix)
        transformed = [x for x in transformed]
        difference = [a - b for a, b in zip(transformed, allen0)]
        ss = sum_square_com(difference)
        error.append(ss)
    bad_keys = set(common_keys) - set(good_keys)
    print(f'RMS {threshold} keys remove={bad_keys} error={round(sum(error)/len(common_keys),2)}')
#

RMS 100 keys remove={'3N_R', 'VLL_R', 'SNR_R', 'Sp5I_R', 'SNC_R', 'VLL_L', '4N_R', '7n_L', 'Sp5I_L', 'DC_L', '5N_L', '7N_R', 'Sp5C_L', 'AP', '3N_L', 'SNC_L', '6N_R', '7N_L', 'LC_L', 'Sp5O_R', 'SNR_L', '5N_R', 'Amb_L', 'Amb_R', 'PBG_L', 'RtTg', '7n_R', '4N_L', 'DC_R', 'LC_R', 'PBG_R', 'Sp5O_L', '6N_L'} error=276.55
RMS 125 keys remove={'3N_R', 'VLL_R', 'SNR_R', 'Sp5I_R', 'SNC_R', 'VLL_L', '4N_R', '7n_L', 'Sp5I_L', 'DC_L', '5N_L', '7N_R', 'Sp5C_L', 'AP', '3N_L', 'SNC_L', '6N_R', '7N_L', 'LC_L', 'Sp5O_R', 'SNR_L', 'Amb_L', 'Amb_R', 'PBG_L', 'RtTg', '7n_R', '4N_L', 'DC_R', 'LC_R', 'PBG_R', 'Sp5O_L', '6N_L'} error=253.98
RMS 150 keys remove={'VLL_R', 'SNR_R', 'Sp5I_R', 'VLL_L', '4N_R', '7n_L', 'Sp5I_L', 'DC_L', '7N_R', 'Sp5C_L', 'AP', 'SNC_L', '7N_L', 'LC_L', 'Sp5O_R', 'SNR_L', 'Amb_R', 'Amb_L', 'PBG_L', 'RtTg', '7n_R', '4N_L', 'DC_R', 'LC_R', 'PBG_R', 'Sp5O_L'} error=254.83
RMS 175 keys remove={'VLL_R', 'SNR_R', 'Sp5I_R', 'VLL_L', '4N_R', '7n_L', 'Sp5I_L', 'DC_L', '7N_R', 'Sp5C_L', 'AP', '